To import the raw data in FITS format, I will use the [Astropy](https://www.astropy.org) package.

In [9]:
# pip install astropy lxml

In [10]:
from astropy.io import fits

In [11]:
import os
import requests

base_url = "https://umbra.nascom.nasa.gov/goes/fits"

- As we will see, importing even one year of data is challenging. 
- For this example I will focus on the year 2018.

In [ ]:
Important

In [12]:
import pandas as pd
import numpy as np
year = 2018
df = pd.read_html(f"{base_url}/{year}")[0][["Name"]].dropna()
files = df["Name"][df["Name"].str.endswith("fits")]

In [3]:
f = fits.open('fits/go1020050101.fits')

In [18]:
f["FLUXES"].data["FLUX"]

array([[[1.2839962e-07, 5.1667759e-09],
        [1.2717541e-07, 5.0859854e-09],
        [1.2924733e-07, 5.0656310e-09],
        ...,
        [1.8686031e-07, 3.8584749e-09],
        [1.8686031e-07, 3.4860725e-09],
        [1.8685836e-07, 3.4859755e-09]]], dtype='>f4')

In [57]:
import urllib.request
xl = []
for file in files:
    month = int(file[:-5][-4:-2])
    day = int(file[:-5][-2:])
    urllib.request.urlretrieve(f"{base_url}/{year}/{file}", file)
    with fits.open(file) as io:
        fld = np.asarray(io["FLUXES"].data)
        # The time unit is seconds from midnight UTC, observations
        # are approximately once every two seconds
        tim = fld["Time"][0,:]
        # The flux levels in watts per square meter
        flx = fld["Flux"][0,:,:]
        x = np.hstack((tim[:, None], flx))
        x = pd.DataFrame(x, columns=["Time", "Flux1", "Flux2"])
        x["Year"] = year
        x["Month"] = month
        x["Day"] = day
        x = x[["Year", "Month", "Day", "Time", "Flux1", "Flux2"]]
        xl.append(x)

In [13]:
df = pd.concat(xl, axis=0)
df

NameError: name 'xl' is not defined

In [74]:
df12 = df[df["Month"] == 12]
df12.to_csv("goes_2018_12.csv.gz")